<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/RNA3D_Grok3_deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+-----------------------+ | 輸入測試數據 | | (CSV格式含id,sequence)| +----------|------------+ ▼ +-----------------------+ | 數據預處理模組 | # 轉換T→U/大寫 +----------|------------+ ▼ +-----------------------+ | 動態規劃矩陣計算 | # 填滿Nussinov矩陣 +----------|------------+ ▼ +-----------------------+ | 二級結構回溯解析 | # 從矩陣提取配對 +----------|------------+ ▼ +-----------------------+ | 格式轉換與輸出 | # 生成點括號格式 +-----------------------+

In [10]:
import pandas as pd
from functools import lru_cache

class RNA結構預測器:
    def __init__(self):
        self.配對規則 = {('A', 'U'), ('U', 'A'), ('G', 'C'), ('C', 'G'), ('G', 'U'), ('U', 'G')}

    def 驗證序列(self, 序列):
        序列 = 序列.upper().replace('T', 'U')
        合法核苷酸 = {'A', 'U', 'C', 'G'}
        assert all(c in 合法核苷酸 for c in 序列), "含無效核苷酸"
        return 序列

    def 預測(self, 序列):
        try:
            有效序列 = self.驗證序列(序列)
            n = len(有效序列)

            @lru_cache(maxsize=None)
            def dp(i, j):
                if i >= j - 3:  # 至少相隔4個碱基（j-i-1 >= 4 → j-i >= 4）
                    return 0
                max_pairs = dp(i+1, j)  # 跳過i
                max_pairs = max(max_pairs, dp(i, j-1))  # 跳過j

                # 檢查i和j是否可以配對
                if (有效序列[i], 有效序列[j]) in self.配對規則:
                    max_pairs = max(max_pairs, dp(i+1, j-1) + 1)

                # 檢查所有可能的分割點k
                for k in range(i, j):
                    current = dp(i, k) + dp(k+1, j)
                    if current > max_pairs:
                        max_pairs = current
                return max_pairs

            return dp(0, n-1)
        except AssertionError as e:
            print(f"錯誤: {e}")
            raise  # 重新拋出異常以便測試案例捕獲
        except Exception as e:
            print(f"發生錯誤: {e}")
            raise

# 測試案例
if __name__ == "__main__":
    預測器 = RNA結構預測器()

    # 測試有效序列
    try:
        result = 預測器.預測('AUGCT')
        print(f"測試1結果: {result}")
        assert result == 1  # 示例預期值，需根據實際情況調整
    except Exception as e:
        print(f"測試1失敗: {e}")

    # 測試無效字符
    try:
        預測器.預測('SAUCE')
        print("測試2失敗: 未拋出異常")
    except AssertionError as e:
        print(f"測試2通過: {e}")
    except Exception as e:
        print(f"測試2異常: {e}")

    # 測試短序列
    try:
        result = 預測器.預測('A')
        print(f"測試3結果: {result}")
        assert result == 0
    except Exception as e:
        print(f"測試3失敗: {e}")

測試1結果: 1
錯誤: 含無效核苷酸
測試2通過: 含無效核苷酸
測試3結果: 0
